In [ ]:
import requests, pandas as pd, urllib3, bisect
from datetime import datetime, timezone

def smard_range(filter_id="410", region="DE", resolution="quarterhour",
                start="2025-11-01", end="2025-11-11",
                base="https://www.smard.de/app/chart_data", verify=False):
    # corp TLS hack (set verify to a PEM bundle in prod)
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    headers = {"User-Agent": "Mozilla/5.0"}

    # parse start/end (accept str or datetime); timestamps are UTC ms
    if isinstance(start, str): start = pd.to_datetime(start, utc=True)
    if isinstance(end, str):   end   = pd.to_datetime(end,   utc=True)
    if end < start: raise ValueError("end must be >= start")
    start_ms = int(start.timestamp()*1000)
    end_ms   = int(end.timestamp()*1000)

    # 1) list chunk timestamps
    idx = requests.get(f"{base}/{filter_id}/{region}/index_{resolution}.json",
                       headers=headers, timeout=60, verify=verify).json()
    stamps = sorted(idx.get("timestamps", []))
    if not stamps: return pd.DataFrame(columns=["time_utc","value"])

    # 2) choose the chunks that cover [start, end]
    i = bisect.bisect_right(stamps, start_ms) - 1  # last stamp <= start
    i = max(i, 0)
    selected = [s for s in stamps[i:] if s <= end_ms]
    if not selected and stamps[i] <= end_ms:
        selected = [stamps[i]]  # at least include the chunk containing start

    # 3) fetch, merge, dedupe (last value wins)
    rows = []
    for ts in selected:
        j = requests.get(f"{base}/{filter_id}/{region}/{filter_id}_{region}_{resolution}_{ts}.json",
                         headers=headers, timeout=60, verify=verify).json()
        rows += j.get("series") or j.get("series2") or []
    if not rows: return pd.DataFrame(columns=["time_utc","value"])

    data = sorted({int(t): v for t, v in rows}.items())
    df = pd.DataFrame(data, columns=["epoch_ms","value"])
    df["time_utc"] = pd.to_datetime(df["epoch_ms"], unit="ms", utc=True)

    # 4) precise time window filter
    m = (df["epoch_ms"] >= start_ms) & (df["epoch_ms"] <= end_ms)
    return df.loc[m, ["time_utc","value"]].reset_index(drop=True)

In [7]:
# DE/LU day-ahead price, hourly, Oct 2025
df = smard_range(filter_id="4071", resolution="quarterhour",
                 start="2025-10-01", end="2025-10-31", verify=False)
print(df.head(), df.tail())

                   time_utc    value
0 2025-10-01 00:00:00+00:00  2420.25
1 2025-10-01 00:15:00+00:00  2431.50
2 2025-10-01 00:30:00+00:00  2435.25
3 2025-10-01 00:45:00+00:00  2432.75
4 2025-10-01 01:00:00+00:00  2484.00                       time_utc    value
2876 2025-10-30 23:00:00+00:00  1354.34
2877 2025-10-30 23:15:00+00:00  1337.51
2878 2025-10-30 23:30:00+00:00  1333.54
2879 2025-10-30 23:45:00+00:00  1321.99
2880 2025-10-31 00:00:00+00:00  1246.49


In [9]:
from pathlib import Path  
df.to_csv(path_or_buf=Path('C:\Users\Deram1J\OneDrive - EDF\Joakim Documents\Github Files\Trading-\fetch_power'))

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (179584510.py, line 2)

In [ ]:
###################################################
#What each group means

#Stromerzeugung = power generation (by source)

#1223 Lignite (brown coal)

#1224 Nuclear

#1225 Wind offshore

#1226 Hydropower

#1227 Other conventional

#1228 Other renewables

#4066 Biomass

#4067 Wind onshore

#4068 Photovoltaic (solar PV)

#4069 Hard coal

#4070 Pumped storage (generation side)

#4071 Natural gas

#Stromverbrauch = power consumption

#410 Total consumption / grid load (“Netzlast”)

#4359 Residual load (load minus non-dispatchable renewables)

#4387 Pumped storage (consumption side, when pumping)

#Marktpreis = market price (bidding zones / countries)

#4169 Germany/Luxembourg (DE/LU)

#5078 Neighbours of DE/LU (aggregate)

#4996 Belgium

#4997 Norway 2 (NO2 zone)

#4170 Austria

#252 Denmark 1 (DK1)

#253 Denmark 2 (DK2)

#254 France

#255 Italy (North)

#256 Netherlands

#257, 258 Poland (SMARD lists two Poland series; treat both as Polish price series—use the one that fits your needs)

#259 Switzerland

#260 Slovenia

#261 Czechia

#262 Hungary

#Prognostizierte Erzeugung = forecast generation

#3791 Offshore wind (forecast)

#123 Onshore wind (forecast)

#125 Photovoltaic (forecast)

#715 Other (forecast)

#5097 Wind + PV (combined forecast)

#122 Total generation (forecast)
#####################################################